In [12]:
import pandas as pd
import json

# Merge trips and stop_times
tripsStops = pd.merge(trips, stop_times, on="trip_id", how="left")

# Group by stop_id and collect unique route_ids
stopsToRoutes = (
    tripsStops.groupby("stop_id")["route_id"]
    .apply(lambda x: list(set(x)))
    .reset_index()
)

# Convert to dictionary
stopsToRoutesDict = dict(zip(stopsToRoutes["stop_id"], stopsToRoutes["route_id"]))




In [13]:
print(stopsToRoutesDict[64285])

[102699]


In [22]:
import pandas as pd
import folium
import random
import json
from branca.element import MacroElement, Element
from jinja2 import Template
from collections import defaultdict
from folium.plugins import Draw

# --- 1. Load GTFS Data ---
# Ensure your GTFS .txt files are in a subdirectory named 'data'.
try:
    shapes = pd.read_csv("data/shapes.txt")
    stops = pd.read_csv("data/stops.txt")
    trips = pd.read_csv("data/trips.txt")
    routes = pd.read_csv("data/routes.txt")
    stop_times = pd.read_csv("data/stop_times.txt")
except FileNotFoundError as e:
    print(f"Error loading data: {e}. Make sure the GTFS files are in a 'data/' directory.")
    exit()

# --- 2. Preprocess Data ---
trip_route_map = trips[['trip_id', 'route_id', 'shape_id']].drop_duplicates()
route_names = routes[['route_id', 'route_short_name', 'route_long_name']]
shape_route_map = pd.merge(trip_route_map, route_names, on='route_id', how='left').drop_duplicates(subset=['shape_id'])
stops_dict = stops.set_index('stop_id').to_dict('index')
trip_to_stops_df = stop_times.groupby('trip_id')['stop_id'].apply(list).reset_index()

route_data = {}
for _, row in shape_route_map.iterrows():
    shape_id = row['shape_id']
    route_name = row['route_short_name'] if pd.notna(row['route_short_name']) else row['route_long_name']
    route_id = row['route_id']
    
    trips_for_shape = trip_route_map[trip_route_map['shape_id'] == shape_id]
    if trips_for_shape.empty: continue
    
    rep_trip_id = trips_for_shape['trip_id'].iloc[0]
    stop_ids_series = trip_to_stops_df.loc[trip_to_stops_df['trip_id'] == rep_trip_id, 'stop_id']
    stop_ids = stop_ids_series.iloc[0] if not stop_ids_series.empty else []

    shape_points = shapes[shapes['shape_id'] == shape_id].sort_values('shape_pt_sequence')
    path = shape_points[['shape_pt_lat', 'shape_pt_lon']].values.tolist()
    
    stops_on_route = []
    for sid in stop_ids:
        stop_info = stops_dict.get(sid)
        if stop_info:
            stops_on_route.append({
                'lat': stop_info['stop_lat'], 'lon': stop_info['stop_lon'],
                'name': stop_info['stop_name'], 'id': str(sid)  # Convert to string for consistent typing
            })
            
    route_data[route_id] = {
        'route_name': route_name, 'color': f'#{random.randint(0, 0xFFFFFF):06x}',
        'shape': path, 'stops': stops_on_route
    }

# Create a comprehensive stops lookup for JavaScript
stops_lookup = {}
for _, stop in stops.iterrows():
    key = f"{stop['stop_lat']:.6f},{stop['stop_lon']:.6f}"
    stops_lookup[key] = {
        'stop_id': str(stop['stop_id']),  # Ensure stop_id is string
        'stop_name': stop['stop_name'],
        'lat': stop['stop_lat'],
        'lon': stop['stop_lon']
    }

stop_to_routes_map = stopsToRoutesDict

# Convert to regular dict for proper JSON serialization
stop_to_routes_map = dict(stop_to_routes_map)

# --- 3. Initialize the Folium Map ---
center = [stops['stop_lat'].mean(), stops['stop_lon'].mean()]
m = folium.Map(location=center, zoom_start=12)

# Layer order: routes first, then stops on top
highlighted_routes_fg = folium.FeatureGroup(name="Highlighted Routes", show=True).add_to(m)
drawn_fg = folium.FeatureGroup(name="Drawn Area", show=True).add_to(m)
centroid_fg = folium.FeatureGroup(name="Shape Centroids", show=True).add_to(m)
all_stops_fg = folium.FeatureGroup(name="All Stops", show=True).add_to(m)

for _, stop in stops.iterrows():
    marker = folium.CircleMarker(
        location=(stop['stop_lat'], stop['stop_lon']), radius=3, color='blue',
        fill_color='blue', fill_opacity=0.7,
        popup=f"<b>{stop['stop_name']}</b><br>Stop ID: {stop['stop_id']}"
    )
    # Store stop_id as string for consistent access
    marker.options['stop_id'] = str(stop['stop_id'])
    marker.add_to(all_stops_fg)

# --- 4. Add Map Controls and UI Elements ---
draw = Draw(
    draw_options={
        "polyline": False, "polygon": False, "circlemarker": False, "marker": False,
        "rectangle": {"shapeOptions": {"color": "#007BFF", "fillOpacity": 0.1}},
        "circle": {"shapeOptions": {"color": "#28A745", "fillOpacity": 0.1}}
    },
    edit_options={"edit": False}
).add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

# --- 5. Custom HTML and JavaScript ---

# A. HTML Control Panel
dropdown_options = "".join(
    f'<option value="{rid}">{val["route_name"]}</option>' for rid, val in route_data.items()
)
control_html = f"""
<div id="controlPanel" style="position: fixed; top: 10px; left: 50px; z-index:999; background: white; 
                            padding: 10px; border-radius: 6px; box-shadow: 2px 2px 5px rgba(0,0,0,0.3);
                            font-family: Arial, sans-serif; max-width: 280px;">
    <h4 style="margin-top: 0;">Route Explorer</h4>
    <label for="routeDropdown"><strong>Select a Route:</strong></label><br>
    <select id="routeDropdown" style="width: 100%; margin-top: 5px;">
        <option value="">-- Show All Stops --</option>
        {dropdown_options}
    </select>
    <hr style="margin: 15px 0;">
    <label><strong>Go to Location:</strong></label>
    <input type="text" id="latInput" placeholder="Latitude" size="10" style="width: 95%; margin-top: 5px;"/>
    <input type="text" id="lonInput" placeholder="Longitude" size="10" style="width: 95%; margin-top: 5px;"/>
    <button onclick="window.mapInteraction.goToLocation()" 
            style="width: 100%; background-color: #28a745; color: white; padding: 8px 0; border: none; 
                   border-radius: 4px; cursor: pointer; margin-top: 5px;">Go</button>
    <hr style="margin: 15px 0;">
    <button onclick="window.mapInteraction.clearAll()" 
            style="width: 100%; background-color: #dc3545; color: white; padding: 10px 0; border: none; 
                   border-radius: 4px; font-size: 14px; cursor: pointer;">
      Clear Drawings & Highlights
    </button>
    
    <!-- Shape Info Display -->
    <div id="shapeInfo" style="margin-top: 10px; padding: 8px; background: #e7f3ff; border-radius: 4px; border-left: 4px solid #007BFF; display: none;">
        <strong>Shape Information:</strong><br>
        <div id="shapeDetails" style="font-size: 12px; margin-top: 5px;"></div>
    </div>
    
    <div id="debugInfo" style="margin-top: 10px; padding: 5px; background: #f0f0f0; border-radius: 3px; font-size: 12px; display: none;">
        <strong>Debug Info:</strong><br>
        <span id="debugText"></span>
    </div>
</div>
"""
m.get_root().html.add_child(Element(control_html))

# B. JavaScript Template
# Update the JavaScript Template section with this new version
# Update the JavaScript Template section with this new version
class MapInteraction(MacroElement):
    _template = Template(u"""
        {% macro script(this, kwargs) %}
            var map = {{this._parent.get_name()}};
            var allStopsFG = {{this.all_stops_fg.get_name()}};
            var drawnFG = {{this.drawn_fg.get_name()}};
            var centroidFG = {{this.centroid_fg.get_name()}};
            var highlightedRoutesFG = {{this.highlighted_routes_fg.get_name()}};
            
            var routeData = {{this.route_data_json | safe}};
            var stopToRoutesMap = {{this.stop_to_routes_map_json | safe}};
            var stopsLookup = {{this.stops_lookup_json | safe}};
            
            var selectedRouteLayer = null;
            var selectedRouteStopMarkers = [];
            var goToMarker = null;
            var highlightedRoutes = [];
            var currentCentroidMarker = null;
            var currentShapeTooltip = null;
            var circumscribedRectangle = null;

            function calculateDistance(lat1, lon1, lat2, lon2) {
                var R = 6371000;
                var dLat = (lat2 - lat1) * Math.PI / 180;
                var dLon = (lon2 - lon1) * Math.PI / 180;
                var a = Math.sin(dLat/2) * Math.sin(dLat/2) +
                        Math.cos(lat1 * Math.PI / 180) * Math.cos(lat2 * Math.PI / 180) *
                        Math.sin(dLon/2) * Math.sin(dLon/2);
                var c = 2 * Math.atan2(Math.sqrt(a), Math.sqrt(1-a));
                return R * c;
            }

            function formatDistance(meters) {
                if (meters < 1000) return meters.toFixed(1) + ' m';
                return (meters / 1000).toFixed(2) + ' km';
            }

            function createCircumscribedRectangle(circle) {
                var center = circle.getLatLng();
                var radius = circle.getRadius();
                var earthRadius = 6371000;
                var angularDistance = radius / earthRadius * (180 / Math.PI);
                
                var north = center.lat + angularDistance;
                var south = center.lat - angularDistance;
                var east = center.lng + angularDistance / Math.cos(center.lat * Math.PI / 180);
                var west = center.lng - angularDistance / Math.cos(center.lat * Math.PI / 180);
                
                return L.rectangle([[south, west], [north, east]], {
                    color: '#ff7800',
                    weight: 2,
                    fillOpacity: 0.1,
                    dashArray: '5, 5'
                });
            }

            function getRectangleInfo(bounds) {
                var north = bounds.getNorth();
                var south = bounds.getSouth();
                var east = bounds.getEast();
                var west = bounds.getWest();
                
                var latSpan = Math.abs(north - south);
                var lonSpan = Math.abs(east - west);
                
                var latSpanDistance = calculateDistance(south, west, north, west);
                var lonSpanDistance = calculateDistance(south, west, south, east);
                
                return {
                    north: north.toFixed(6),
                    south: south.toFixed(6),
                    east: east.toFixed(6),
                    west: west.toFixed(6),
                    latSpan: latSpan.toFixed(6),
                    lonSpan: lonSpan.toFixed(6),
                    latSpanDistance: formatDistance(latSpanDistance),
                    lonSpanDistance: formatDistance(lonSpanDistance)
                };
            }

            function updateShapeInfo(shapeData) {
                var shapeInfo = document.getElementById('shapeInfo');
                var shapeDetails = document.getElementById('shapeDetails');
                shapeInfo.style.display = shapeData ? 'block' : 'none';
                if (shapeData) shapeDetails.innerHTML = shapeData;
            }

            function updateDebugInfo(text) {
                var debugDiv = document.getElementById('debugInfo');
                var debugText = document.getElementById('debugText');
                if (debugDiv && debugText) {
                    debugText.innerHTML = text;
                    debugDiv.style.display = 'block';
                }
            }

            function processStopsInShape(shape) {
                var stopsInShape = [];
                var shapeType = shape instanceof L.Circle ? "circle" : 
                               shape instanceof L.Rectangle ? "rectangle" : "unknown";
                
                allStopsFG.eachLayer(function(stop) {
                    var stopLatLng = stop.getLatLng();
                    var isInside = false;
                    
                    try {
                        if (shapeType === "circle") {
                            isInside = shape.getLatLng().distanceTo(stopLatLng) <= shape.getRadius();
                        } else if (shapeType === "rectangle") {
                            isInside = shape.getBounds().contains(stopLatLng);
                        }
                    } catch (e) {
                        console.error("Error checking stop location:", e);
                    }
                    
                    if (isInside) {
                        stop.setStyle({color: 'red', fillColor: 'red', fillOpacity: 1.0, radius: 6});
                        var coordKey = stopLatLng.lat.toFixed(6) + "," + stopLatLng.lng.toFixed(6);
                        var stopInfo = stopsLookup[coordKey];
                        stopsInShape.push(stopInfo?.stop_id || stop.options.stop_id || stop.stop_id);
                    } else {
                        stop.setStyle({color: 'blue', fillColor: 'blue', fillOpacity: 0.7, radius: 3});
                    }
                });
                
                return stopsInShape;
            }

            function highlightRoutesForStops(stopsInShape) {
                highlightedRoutesFG.clearLayers();
                highlightedRoutes.forEach(function(route) {
                    if (map.hasLayer(route)) map.removeLayer(route);
                });
                highlightedRoutes = [];
                
                if (stopsInShape.length === 0) {
                    updateDebugInfo("No stops found in shape");
                    return;
                }
                
                var routesToHighlight = new Set();
                var routeDetails = [];
                
                stopsInShape.forEach(function(stopId) {
                    if (stopToRoutesMap[stopId]) {
                        stopToRoutesMap[stopId].forEach(function(routeId) {
                            routesToHighlight.add(routeId);
                        });
                    }
                });
                
                routesToHighlight.forEach(function(routeId) {
                    var data = routeData[routeId];
                    if (data?.shape?.length > 0) {
                        try {
                            var routeLine = L.polyline(data.shape, {
                                color: data.color,
                                weight: 8,
                                opacity: 0.9,
                                dashArray: '10, 5'
                            }).addTo(highlightedRoutesFG);
                            
                            highlightedRoutes.push(routeLine);
                            routeDetails.push(
                                "Route " + routeId + ": " + (data.route_name || 'Unnamed Route')
                            );
                            
                            routeLine.bindPopup(
                                "<b>Route: " + (data.route_name || 'Unnamed') + "</b><br>" +
                                "Route ID: " + routeId
                            );
                        } catch (e) {
                            console.error("Error creating route line:", e);
                        }
                    }
                });
                
                updateDebugInfo(
                    "Highlighting " + routesToHighlight.size + " routes:<br>" +
                    "<ul><li>" + routeDetails.join("</li><li>") + "</li></ul>"
                );
            }

            function setupStopClickHandlers() {
                allStopsFG.eachLayer(function(stop) {
                    stop.off('click').on('click', function(e) {
                        var latLng = e.latlng;
                        var coordKey = latLng.lat.toFixed(6) + "," + latLng.lng.toFixed(6);
                        var stopInfo = stopsLookup[coordKey] || {};
                        
                        var popupContent = "<b>" + (stopInfo.stop_name || 'Unknown Stop') + "</b><br>" +
                                          "Stop ID: " + (stopInfo.stop_id || 'N/A') + "<br>" +
                                          "Latitude: " + latLng.lat.toFixed(6) + "<br>" +
                                          "Longitude: " + latLng.lng.toFixed(6);
                        
                        if (stopInfo.stop_id && stopToRoutesMap[stopInfo.stop_id]) {
                            popupContent += "<br><b>Served by:</b> " + 
                                stopToRoutesMap[stopInfo.stop_id].map(function(routeId) {
                                    return routeData[routeId]?.route_name || routeId;
                                }).join(", ");
                        }
                        
                        stop.bindPopup(popupContent).openPopup();
                        
                        updateDebugInfo(
                            "Clicked stop: " + (stopInfo.stop_name || 'Unknown') + "<br>" +
                            "ID: " + (stopInfo.stop_id || 'N/A') + "<br>" +
                            "Position: " + latLng.lat.toFixed(6) + ", " + latLng.lng.toFixed(6)
                        );
                    });
                });
            }

            window.mapInteraction = {
                resetStopStyles: function() {
                    allStopsFG.eachLayer(function(stop) {
                        stop.setStyle({color: 'blue', fillColor: 'blue', fillOpacity: 0.7, radius: 3});
                    });
                },

                clearAll: function() {
                    drawnFG.clearLayers();
                    centroidFG.clearLayers();
                    highlightedRoutesFG.clearLayers();
                    if (currentCentroidMarker) map.removeLayer(currentCentroidMarker);
                    if (currentShapeTooltip) map.removeLayer(currentShapeTooltip);
                    if (circumscribedRectangle) map.removeLayer(circumscribedRectangle);
                    if (goToMarker) map.removeLayer(goToMarker);
                    if (selectedRouteLayer) map.removeLayer(selectedRouteLayer);
                    
                    this.resetStopStyles();
                    selectedRouteStopMarkers.forEach(m => map.removeLayer(m));
                    highlightedRoutes.forEach(r => map.hasLayer(r) && map.removeLayer(r));
                    
                    selectedRouteStopMarkers = [];
                    highlightedRoutes = [];
                    currentCentroidMarker = null;
                    currentShapeTooltip = null;
                    circumscribedRectangle = null;
                    
                    document.getElementById('routeDropdown').value = "";
                    updateShapeInfo(null);
                    updateDebugInfo("Cleared all layers");
                },

                goToLocation: function() {
                    var lat = parseFloat(document.getElementById('latInput').value);
                    var lon = parseFloat(document.getElementById('lonInput').value);
                    if (isNaN(lat) || isNaN(lon)) {
                        alert("Invalid coordinates!");
                        return;
                    }
                    if (goToMarker) map.removeLayer(goToMarker);
                    map.flyTo([lat, lon], 16);
                    goToMarker = L.marker([lat, lon])
                        .bindPopup('<b>Location</b><br>Lat: ' + lat + '<br>Lon: ' + lon)
                        .addTo(map)
                        .openPopup();
                },

                highlightFromShape: function(drawnLayer) {
                    if (circumscribedRectangle) map.removeLayer(circumscribedRectangle);
                    
                    if (drawnLayer instanceof L.Circle) {
                        circumscribedRectangle = createCircumscribedRectangle(drawnLayer).addTo(drawnFG);
                        var rectInfo = getRectangleInfo(circumscribedRectangle.getBounds());
                        circumscribedRectangle.bindPopup(
                            '<b>Circumscribed Rectangle</b><br>' +
                            'North: ' + rectInfo.north + '<br>' +
                            'South: ' + rectInfo.south + '<br>' +
                            'East: ' + rectInfo.east + '<br>' +
                            'West: ' + rectInfo.west + '<br>' +
                            'Lat Span: ' + rectInfo.latSpan + '° (' + rectInfo.latSpanDistance + ')<br>' +
                            'Lon Span: ' + rectInfo.lonSpan + '° (' + rectInfo.lonSpanDistance + ')'
                        );
                        highlightRoutesForStops(processStopsInShape(circumscribedRectangle));
                    } else if (drawnLayer instanceof L.Rectangle) {
                        var rectInfo = getRectangleInfo(drawnLayer.getBounds());
                        drawnLayer.bindPopup(
                            '<b>Rectangle</b><br>' +
                            'North: ' + rectInfo.north + '<br>' +
                            'South: ' + rectInfo.south + '<br>' +
                            'East: ' + rectInfo.east + '<br>' +
                            'West: ' + rectInfo.west + '<br>' +
                            'Lat Span: ' + rectInfo.latSpan + '° (' + rectInfo.latSpanDistance + ')<br>' +
                            'Lon Span: ' + rectInfo.lonSpan + '° (' + rectInfo.lonSpanDistance + ')'
                        );
                        highlightRoutesForStops(processStopsInShape(drawnLayer));
                    }
                    
                    this.addCentroidMarker(drawnLayer);
                },

                addCentroidMarker: function(drawnLayer) {
                    if (currentCentroidMarker) map.removeLayer(currentCentroidMarker);
                    
                    var centroid, shapeInfo = "";
                    if (drawnLayer instanceof L.Circle) {
                        centroid = drawnLayer.getLatLng();
                        shapeInfo = "Circle<br>Radius: " + formatDistance(drawnLayer.getRadius());
                    } else if (drawnLayer.getBounds) {
                        centroid = drawnLayer.getBounds().getCenter();
                        var rectInfo = getRectangleInfo(drawnLayer.getBounds());
                        shapeInfo = "Rectangle<br>" +
                                   "Lat Span: " + rectInfo.latSpan + "° (" + rectInfo.latSpanDistance + ")<br>" +
                                   "Lon Span: " + rectInfo.lonSpan + "° (" + rectInfo.lonSpanDistance + ")";
                    }
                    
                    if (centroid) {
                        currentCentroidMarker = L.marker([centroid.lat, centroid.lng], {
                            icon: L.divIcon({
                                className: 'centroid-marker',
                                html: '<div style="background: red; width: 12px; height: 12px; border-radius: 50%; border: 2px solid white;"></div>',
                                iconSize: [12, 12]
                            })
                        }).bindPopup('<b>Centroid</b><br>' + centroid.lat.toFixed(6) + ', ' + centroid.lng.toFixed(6))
                          .addTo(centroidFG);
                        updateShapeInfo(shapeInfo);
                    }
                },

                displaySelectedRoute: function(rid) {
                    if (selectedRouteLayer) map.removeLayer(selectedRouteLayer);
                    selectedRouteStopMarkers.forEach(m => map.removeLayer(m));
                    selectedRouteStopMarkers = [];
                    this.resetStopStyles();
                    
                    if (!rid) return;
                    
                    var data = routeData[rid];
                    if (!data) return;
                    
                    if (data.shape?.length > 0) {
                        selectedRouteLayer = L.polyline(data.shape, {
                            color: data.color,
                            weight: 10,
                            opacity: 1.0
                        }).bindPopup("<b>" + (data.route_name || 'Unnamed Route') + "</b><br>ID: " + rid)
                          .addTo(map);
                        
                        if (data.stops) {
                            data.stops.forEach(function(stop) {
                                var marker = L.circleMarker([stop.lat, stop.lon], {
                                    radius: 8, 
                                    color: 'white', 
                                    weight: 2, 
                                    fillColor: data.color, 
                                    fillOpacity: 1
                                }).bindPopup("<b>" + stop.name + "</b><br>ID: " + stop.id)
                                  .addTo(map);
                                selectedRouteStopMarkers.push(marker);
                            });
                        }
                        
                        map.fitBounds(selectedRouteLayer.getBounds());
                    }
                }
            };

            // Initialize event handlers
            setupStopClickHandlers();
            
            map.on('draw:created', function(e) {
                drawnFG.addLayer(e.layer);
                window.mapInteraction.highlightFromShape(e.layer);
            });
            
            map.on('draw:deleted', function() {
                window.mapInteraction.clearAll();
            });
            
            document.getElementById('routeDropdown').addEventListener('change', function() {
                window.mapInteraction.displaySelectedRoute(this.value);
            });
            
            updateDebugInfo("Ready to explore routes");
        {% endmacro %}
    """)

    def __init__(self, all_stops_fg, drawn_fg, centroid_fg, highlighted_routes_fg, route_data, stop_to_routes_map, stops_lookup):
        super(MapInteraction, self).__init__()
        self._name = 'MapInteraction'
        self.all_stops_fg = all_stops_fg
        self.drawn_fg = drawn_fg
        self.centroid_fg = centroid_fg
        self.highlighted_routes_fg = highlighted_routes_fg
        self.route_data_json = json.dumps(route_data)
        self.stop_to_routes_map_json = json.dumps(stop_to_routes_map)
        self.stops_lookup_json = json.dumps(stops_lookup)       
# Add the JavaScript logic to the map
m.add_child(MapInteraction(all_stops_fg, drawn_fg, centroid_fg, highlighted_routes_fg, route_data, stop_to_routes_map, stops_lookup))

# --- 6. Save the Final Map ---
m.save("routesForLocation/combined_gtfs_map.html")